# Weather Info (temperature, weather, humidity and windspeed) 

In [19]:
import pandas as pd
import numpy as np
import warnings
from sklearn.neighbors import LocalOutlierFactor
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

To obtain dummy weather information.

In [20]:
# To obtain weather data 
import requests
import os
from datetime import datetime


#user_api = os.environ['	weather']
api_key = '46eb4ea0e8fadb82e97f406034d68ce5'
city_name = input("Enter the city name: ")

complete_api_link = "https://api.openweathermap.org/data/2.5/weather?q="+city_name+"&appid="+api_key
api_link = requests.get(complete_api_link)
api_data = api_link.json()

if api_data['cod'] == '404':
    print("Invalid City: {}, Please check your City name".format(city_name))
else:
    #create variables to store and display data
    temp_city = ((api_data['main']['temp']) - 273.15)
    weather_desc = api_data['weather'][0]['description']
    hmdt = api_data['main']['humidity']
    wind_spd = api_data['wind']['speed']
    date_time = datetime.now().strftime("%d %b %Y | %I:%M:%S %p")

    print ("-------------------------------------------------------------")
    print ("Weather Stats for - {}  || {}".format(city_name.upper(), date_time))
    print ("-------------------------------------------------------------")

    print ("Current temperature is: {:.2f} deg C".format(temp_city))
    print ("Current weather desc  :",weather_desc)
    print ("Current humidity      :",hmdt, '%')
    print ("Current wind speed    :",wind_spd ,'kmph')
    
current_temperature = temp_city
current_windspeed   = wind_spd
current_humidity    = hmdt
current_weather     = weather_desc

Enter the city name: kuching
-------------------------------------------------------------
Weather Stats for - KUCHING  || 22 Jan 2022 | 06:31:01 PM
-------------------------------------------------------------
Current temperature is: 29.12 deg C
Current weather desc  : scattered clouds
Current humidity      : 70 %
Current wind speed    : 3.6 kmph


In [21]:
#Let's get the city's coordinates (lat and lon)
url = 'https://api.openweathermap.org/data/2.5/weather?q=' + city_name + '&appid=' + api_key

#Let's parse the Json
req = requests.get(url)
data = req.json()

#Let's get the name, the longitude and latitude
name = data['name']
lon = data['coord']['lon']
lat = data['coord']['lat']

#Let's now use the One Call Api to get the 8 day forecast
#We'll need to exclude the minutely and hourly
exclude = "minute,hourly"
url2 = 'https://api.openweathermap.org/data/2.5/onecall?lat='+str(lat)+'&lon='+str(lon)+'&exclude='+'&appid='+api_key

#Let's now parse the JSON
req2 = requests.get(url2)
data2 = req2.json()

#Let's now get the temp for the day, the night and the weather conditions
days = []
nights = []
descr = []
humidity = []
wind_speed = []

import datetime

#Check for today's day
x = datetime.datetime.now()

if (x.strftime("%a") == 'Mon'):
  days_of_week = ['Mon - Today', 'Tue - Tomorrow', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun','Mon']
elif (x.strftime("%a") == 'Tue'):
  days_of_week = ['Tue - Today', 'Wed - Tomorrow', 'Thu', 'Fri', 'Sat', 'Sun','Mon', 'Tue']
elif (x.strftime("%a") == 'Wed'):
  days_of_week = ['Wed - Today', 'Thu - Tomorrow', 'Fri', 'Sat', 'Sun','Mon', 'Tue', 'Wed']
elif (x.strftime("%a") == 'Thu'):
  days_of_week = ['Thu - Today', 'Fri - Tomorrow', 'Sat', 'Sun','Mon', 'Tue', 'Wed', 'Thu']
elif (x.strftime("%a") == 'Fri'):
  days_of_week = ['Fri - Today', 'Sat - Tomorrow', 'Sun','Mon', 'Tue', 'Wed', 'Thu', 'Fri']
elif (x.strftime("%a") == 'Sat'):
  days_of_week = ['Sat - Today', 'Sun - Tomorrow','Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
elif (x.strftime("%a") == 'Sun'):
  days_of_week = ['Sun - Today','Mon - Tomorrow', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']


#We need to access 'daily'
for i in data2['daily']:
        
        #We notice that the temperature is in Kelvin, so we need to do -273.15 for every datapoint
        
        #Let's start by days
        #Let's round the decimal numbers to 2
        days.append(round(i['temp']['day'] - 273.15,2))
        
        #Nights
        nights.append(round(i['temp']['night'] - 273.15,2))
        
        #Let's now get the weather condition and the description
        #'weather' [0] 'main' + 'weather' [0] 'description'
        descr.append(i['weather'][0]['main'] + ": " +i['weather'][0]['description'])

        humidity.append(i['humidity'])

        wind_speed.append(i['wind_speed'])
        

#Let's now format the output to make it readable
string = f'[ {name} - 8 days forecast]\n'

#Let's now loop for as much days as there available (8 in this case):
for i in range(len(days)):
    
    #We want to print out the day 1 = today and day 2 = tomorrow for reference
    string += f'\nDay {i+1} (' + days_of_week[i] + f')\n'
        
    string += 'Morning    :' + str(days[i]) + '°C' + '\n'
    string += 'Night      :' + str(nights[i]) + '°C' + '\n'
    string += 'Conditions :' + descr[i] + '\n'
    string += 'Humidity   :' + str(humidity[i]) + 'g kg-1' + '\n'
    string += 'Wind Speed :' + str(wind_speed[i]) + 'm sec-1' + '\n'
    
print(string)

[ Kuching - 8 days forecast]

Day 1 (Sat - Today)
Morning    :27.99°C
Night      :25.22°C
Conditions :Rain: light rain
Humidity   :79g kg-1
Wind Speed :2.35m sec-1

Day 2 (Sun - Tomorrow)
Morning    :30.35°C
Night      :24.55°C
Conditions :Rain: light rain
Humidity   :66g kg-1
Wind Speed :2.55m sec-1

Day 3 (Mon)
Morning    :30.42°C
Night      :24.66°C
Conditions :Rain: light rain
Humidity   :63g kg-1
Wind Speed :3.62m sec-1

Day 4 (Tue)
Morning    :27.26°C
Night      :24.22°C
Conditions :Rain: light rain
Humidity   :84g kg-1
Wind Speed :2.88m sec-1

Day 5 (Wed)
Morning    :29.35°C
Night      :24.53°C
Conditions :Rain: light rain
Humidity   :69g kg-1
Wind Speed :2.95m sec-1

Day 6 (Thu)
Morning    :27.61°C
Night      :24.34°C
Conditions :Rain: light rain
Humidity   :76g kg-1
Wind Speed :2.01m sec-1

Day 7 (Fri)
Morning    :26.8°C
Night      :23.75°C
Conditions :Rain: moderate rain
Humidity   :86g kg-1
Wind Speed :1.75m sec-1

Day 8 (Sat)
Morning    :29.46°C
Night      :23.91°C
Conditio

In [23]:
#Export as dummy weather csv (weatherForecast.csv)
def createFrame(days_of_week, days, nights, descr, humidity, wind_speed):    
    # To dataframe
    newframe = {'Index'             : [],
                'Day'               : [],
                'MorningTemp'       : [],
                'NightTemp'         : [],
                'Humidity'          : [],
                'WindSpeed'         : [],
                'Weather'  : []}
    newframe = pd.DataFrame(newframe)
    
    z = 0
    for x in days_of_week:
        temframe = {'Index'             : [str(z+1)],
                    'Day'               : [days_of_week[z]],
                    'MorningTemp'       : [days[z]],
                    'NightTemp'         : [nights[z]],
                    'Humidity'          : [humidity[z]],
                    'WindSpeed'         : [wind_speed[z]],
                    'Weather'           : [descr[z]]}
        z += 1
        temframe = pd.DataFrame(temframe)
        newframe = pd.concat([newframe, temframe])
    
    return newframe

weatherForecast = createFrame(days_of_week, days, nights, descr, humidity, wind_speed)
weatherForecast.to_csv('../dataset/weatherForecast.csv', index = False)
weatherForecast

,Index,Day,MorningTemp,NightTemp,Humidity,WindSpeed,Weather
0,1,Sat - Today,27.99,25.22,79.0,2.35,Rain: light rain
0,2,Sun - Tomorrow,30.35,24.55,66.0,2.55,Rain: light rain
0,3,Mon,30.42,24.66,63.0,3.62,Rain: light rain
0,4,Tue,27.26,24.22,84.0,2.88,Rain: light rain
0,5,Wed,29.35,24.53,69.0,2.95,Rain: light rain
0,6,Thu,27.61,24.34,76.0,2.01,Rain: light rain
0,7,Fri,26.80,23.75,86.0,1.75,Rain: moderate rain
0,8,Sat,29.46,23.91,64.0,2.62,Clouds: scattered clouds


# Climate Suitability Plant Suggestion

In [14]:
# #Export as plant suggestion based on climate csv (weatherForecast.csv)
# def createClimateSuitabilityFrame(days_of_week, days, nights, descr, humidity, wind_speed):    
#     # To dataframe
#     newframe = {'Index'             : [],
#                 'Plant'             : [],
#                 'Germination'       : [],jh
#                 'DaysToMaturity'    : [],
#                 'Harvest'           : []}
#     newframe = pd.DataFrame(newframe)
    
#     z = 0
#     for x in days_of_week:
#         temframe = {'Index'             : [str(z+1)],
#                     'Day'               : [days_of_week[z]],
#                     'MorningTemp'       : [days[z]],
#                     'NightTemp'         : [nights[z]],
#                     'Humidity'          : [humidity[z]],
#                     'WindSpeed'         : [wind_speed[z]],
#                     'Weather'           : [descr[z]]}
#         z += 1
#         temframe = pd.DataFrame(temframe)
#         newframe = pd.concat([newframe, temframe])
    
#     return newframe

# weatherForecast = createFrame(days_of_week, days, nights, descr, humidity, wind_speed)
# weatherForecast.to_csv('../dataset/weatherForecast.csv', index = False)
# weatherForecast

In [15]:
import pandas as pd

#Export as plant suggestion based on climate csv (weatherForecast.csv)
def createClimateSuitabilityFrame():    
    # To dataframe
    newframe = {'Index'             : [],
                'Plant'             : [],
                'Germination'       : [],
                'DaysToMaturity'    : [],
                'Harvest'           : []}
    newframe = pd.DataFrame(newframe)
    
    return newframe

newframe = createClimateSuitabilityFrame()
newframe.to_csv('../dataset/newframe.csv', index = False)
newframe

,Index,Plant,Germination,DaysToMaturity,Harvest


,Index,Link
0,1,https://tableau.dosm.gov.my/t/BPHPP-BahagianPe...


# Agriculture Advise List

In [16]:
# To create dummy agriculture advise list
adviseList = pd.read_csv('../dataset/adviseList.csv')
adviseList.head(5)

# to provide situational advice based on crop selected and their initiation data
# /----- to do
# DaysToMaturity 
# ------/

,Index,Weather,Advise,Precaution,Image
0,0,Rain: light rain,Wear the proper footwear to avoid exposing you...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...
1,1,Rain: light rain,Prepare equipment such as using fabric plant c...,Keep your crops away from rain to avoid microb...,NaN
2,2,Rain: moderate rain,Watch out the water level before going,"Make Sure water areas are fenced like dams, we...",NaN
3,3,Rain: moderate rain,Wear protective equipment to protect ourselves...,Spray animals frequently for removal of ectopa...,NaN
4,4,Rain: heavy rain,Fungicide spray programmes should be maintaine...,Keep an eye out for flare-ups of diseases such...,NaN


# Plant Album

In [14]:
# To create dummy plant album
plantAlbum = pd.read_csv('../dataset/plantAlbum.csv')
# plantAlbum.to_csv('../dataset/plantAlbum.csv', index = True)
# plantAlbum = pd.read_csv('../dataset/plantAlbum.csv')
# plantAlbum = plantAlbum.rename(columns={'Unnamed: 0':'Index'})
# plantAlbum.to_csv('../dataset/plantAlbum.csv', index = False)
plantAlbum.head(5)

,Index,Plant,AlternatePlantName,Image,GrowInstruction,GrowthSpace,HarvestDuration,CompatiblePlant,IncompatiblePlant,AdditionalInfo,Unnamed: 9
0,0,Basil,NaN,https://drive.google.com/file/d/1g5Qt4D5XWCo5m...,"Grow in seed trays, and plant out in 4-6 weeks...",Space plants: 8 - 10 inches apart,Harvest in 10-12 weeks. Pick before flowering.,Compatible with (can grow beside): Tomato,NaN,Basil is commonly used fresh in cooked recipes...,NaN
1,1,Beetroot,Beets,NaN,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 8 - 12 inches apart,Harvest in 7-10 weeks.,"Compatible with (can grow beside): Onions, Sil...","Avoid growing close to: Asparagus, Carrots, Sw...","Apart from boiling whole for salads, beetroot ...",NaN
2,2,Borage,"Burrage, Bugloss",NaN,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 8 inches apart,Harvest in 8-10 weeks. Use leaves before flowe...,"Compatible with (can grow beside): Strawberry,...",NaN,Has a slight cucumber taste which goes well in...,NaN
3,3,Broad beans,Fava bean,NaN,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 6 - 10 inches apart,Harvest in 12-22 weeks. Pick frequently to enc...,"Compatible with (can grow beside): Dill, Potatoes",NaN,The fresh beans are eaten steamed or boiled. A...,Will freeze well. Remove from pods and blanch.
4,4,Broccoli,NaN,NaN,"Easy to grow. Grow in seed trays, and plant ou...",Space plants: 14 - 20 inches apart,Harvest in 10-16 weeks. Cut flowerhead off wit...,Compatible with (can grow beside): Dwarf (bush...,"Avoid growing close to: Climbing (pole) beans,...","The stem (peeled), leaves, and flowerhead are ...",NaN


# Plant Nursary 

In [16]:
# # To create dummy plant nursary log book
# plantNursery = pd.read_csv('../dataset/plantNursery.csv')
# plantNursery

# Agricultural Advisor

To recommend based on dummy weather information.

## Building Advisory Dataframe

In [69]:
# If weatherForecast exist
try:
    df_weather_forecast = pd.read_csv('../dataset/weatherForecast.csv')
except:
    print('weatherForecast.csv File not exist.')
    
# If adviseList exist
try:
    df_advise_list = pd.read_csv('../dataset/adviseList.csv')
except:
    print('adviseList.csv File not exist.')
    
# If stateSuitability exist
try:
    df_state_suitability = pd.read_csv('../dataset/stateSuitability.csv')
except:
    print('stateSuitability.csv File not exist.')
    
# If plantAlbum exist
try:
    df_plant_album = pd.read_csv('../dataset/plantAlbum.csv')
except:
    print('plantAlbum.csv File not exist.')

# # If plantCalender exist
# try:
#     df_plant_calender = pd.read_csv('../dataset/plantCalender.csv')
# except:
#     print('plantCalender.csv File not exist.')
    
    
# Outcomes: weatherAgriAdvise.csv, stateSuitability.csv, cropPriceMalaysia.csv and plantAlbum.csv, 

In [65]:
# Outcome 1: weatherAgriAdvise.csv
weatherAgriAdvise = pd.merge(df_weather_forecast, df_advise_list, on='Weather')
weatherAgriAdvise = weatherAgriAdvise.drop(columns=['Index_y', 'Index_x'])
weatherAgriAdvise.to_csv('../dataset/weatherAgriAdvise.csv', index = False)
weatherAgriAdvise.head(5)

,Day,MorningTemp,NightTemp,Humidity,WindSpeed,Weather,Advise,Precaution,Image
0,Sat - Today,27.99,25.22,79.0,2.35,Rain: light rain,Prepare equipment such as using fabric plant c...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...
1,Sun - Tomorrow,30.35,24.55,66.0,2.55,Rain: light rain,Prepare equipment such as using fabric plant c...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...
2,Mon,30.42,24.66,63.0,3.62,Rain: light rain,Prepare equipment such as using fabric plant c...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...
3,Tue,27.26,24.22,84.0,2.88,Rain: light rain,Prepare equipment such as using fabric plant c...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...
4,Wed,29.35,24.53,69.0,2.95,Rain: light rain,Prepare equipment such as using fabric plant c...,"Protect farm equipment from rain, especially t...",https://drive.google.com/drive/folders/1fCbave...


In [66]:
# Outcome 2: stateSuitability.csv
df_state_suitability.head(5)

,Crop,Johor,Kedah,Kelantan,Melaka,N Sembilan,Pahang,Perak,Perlis,Pinang,Sabah,Sarawak,Selangor,Terengganu
0,Rubber,A,B,A,A,A,A,A,B,A,A,A,A,A
1,Coconut,A,A,A,A,-,A,A,-,A,-,A,A,-
2,Oil-palm,A,A,B,A,A,A,A,B,A,A,A,A,A
3,Cocoa,A,-,-,A,A,A,A,-,-,A,A,A,B
4,Coffee,A,A,A,B,A,A,A,B,A,-,-,A,A


In [67]:
# Outcome 3: cropPriceMalaysia.csv
def createCropPriceFrame():    
    # To dataframe
    CropPrice = {'Index'             : [1],
                 'Link'              : ["https://tableau.dosm.gov.my/t/BPHPP-BahagianPerangkaanhargaPendapatanPerbelanjaan/views/DashboardMyPrice_V3/G_SAYUR?%3AisGuestRedirectFromVizportal=y&%3Aembed=y"]}
    CropPrice = pd.DataFrame(CropPrice)
    
    return CropPrice

CropPrice = createCropPriceFrame()
CropPrice.to_csv('../dataset/cropPriceMalaysia.csv', index = False)
CropPrice.head(5)

,Index,Link
0,1,https://tableau.dosm.gov.my/t/BPHPP-BahagianPe...


In [71]:
# Outcome 4: plantAlbum.csv
df_plant_album.head(5)

,Crop,AlternatePlantName,Image,GrowInstruction,GrowthSpace,HarvestDuration,CompatiblePlant,IncompatiblePlant,AdditionalInfo,Unnamed: 9
0,Basil,NaN,https://drive.google.com/file/d/1g5Qt4D5XWCo5m...,"Grow in seed trays, and plant out in 4-6 weeks...",Space plants: 8 - 10 inches apart,Harvest in 10-12 weeks. Pick before flowering.,Compatible with (can grow beside): Tomato,NaN,Basil is commonly used fresh in cooked recipes...,NaN
1,Beetroot,Beets,https://drive.google.com/file/d/1lplNAD5IyOoG6...,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 8 - 12 inches apart,Harvest in 7-10 weeks.,"Compatible with (can grow beside): Onions, Sil...","Avoid growing close to: Asparagus, Carrots, Sw...","Apart from boiling whole for salads, beetroot ...",NaN
2,Borage,"Burrage, Bugloss",https://drive.google.com/file/d/1dHGaAYRTMODK0...,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 8 inches apart,Harvest in 8-10 weeks. Use leaves before flowe...,"Compatible with (can grow beside): Strawberry,...",NaN,Has a slight cucumber taste which goes well in...,NaN
3,Broad beans,Fava bean,https://drive.google.com/file/d/1XuYX3aSJDIBVf...,Easy to grow. Sow in garden. Sow seed at a dep...,Space plants: 6 - 10 inches apart,Harvest in 12-22 weeks. Pick frequently to enc...,"Compatible with (can grow beside): Dill, Potatoes",NaN,The fresh beans are eaten steamed or boiled. A...,Will freeze well. Remove from pods and blanch.
4,Broccoli,NaN,https://drive.google.com/file/d/1N98A_Zk6Xk7Jf...,"Easy to grow. Grow in seed trays, and plant ou...",Space plants: 14 - 20 inches apart,Harvest in 10-16 weeks. Cut flowerhead off wit...,Compatible with (can grow beside): Dwarf (bush...,"Avoid growing close to: Climbing (pole) beans,...","The stem (peeled), leaves, and flowerhead are ...",NaN


# Crop Production (including nurseries, greenhouses, forestry) 

Advise generation:
1. https://www.epa.gov/agriculture/laws-and-regulations-apply-your-agricultural-operation-farm-activity 
2. https://www.dosh.gov.my/index.php/legislation/guidelines/agriculture-forestry-fishing/609-03-guidelines-on-occupational-safety-and-health-in-agriculture-2002/file 
3.https://www.wri.org/insights/4-ways-farmers-can-adapt-climate-change-and-generate-income 
4.